# Building a Task Manager Agent with Kotlin and ARC

## Setting Up Dependencies

In [1]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("org.eclipse.lmos:arc-langchain4j-client:0.122.0-M2")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.0.0-beta1")

## API Key Configuration

In [2]:
val openAiApiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR-OPENAI-API-KEY"

## Chat Model Configuration

In [3]:
import dev.langchain4j.model.openai.OpenAiChatModel
import org.eclipse.lmos.arc.agents.llm.ChatCompleter
import org.eclipse.lmos.arc.client.langchain4j.LangChainClient
import org.eclipse.lmos.arc.client.langchain4j.LangChainConfig

// Configure the chat provider function that creates a ChatCompleter with GPT-4
val chatProvider : (String?) -> ChatCompleter = {
    LangChainClient(
        languageModel = LangChainConfig(
            modelName = "gpt-4",            // Using GPT-4 as the LLM
            url = null,                     // Using default OpenAI endpoint
            apiKey = openAiApiKey,          // Your API key
            accessKeyId = null,
            secretAccessKey = null,
        ),
        clientBuilder = { config, _ ->
            OpenAiChatModel.builder()
                .modelName(config.modelName)
                .apiKey(config.apiKey)
                .build()
        }
    )
}

## Agent Definition

In [4]:
import org.eclipse.lmos.arc.agents.DSLAgents
import org.eclipse.lmos.arc.agents.dsl.extensions.MemoryScope
import org.eclipse.lmos.arc.agents.dsl.extensions.breakWith
import org.eclipse.lmos.arc.agents.dsl.extensions.memory
import org.eclipse.lmos.arc.agents.llm.ChatCompletionSettings
import org.eclipse.lmos.arc.core.result

val agentBuilder = DSLAgents
    .init(chatProvider)
    .apply {
        define {
            agent {
                name = "task-manager"
                description = "Helps the user manage their tasks: add, remove, list tasks."


                // Core system prompt for the agent's behavior
                prompt {
                    """
                    You are a Task Manager Agent.
                    Your goal is to help the user manage their tasks:
                    they can add tasks, remove tasks, or list tasks.

                    # Instructions
                    - If user wants to add a new task, call the 'add_task' function with the task description.
                    - If user wants to remove a task, call the 'remove_task' function with the exact task name.
                    - If user wants to see all tasks, call the 'list_tasks' function.
                    - If the user asks anything that is not related to tasks, respond with "I only handle tasks."
                    """.trimIndent()
                }


                // Input filter: quickly reject off-topic requests
                // This demonstrates an alternative to handling everything in the prompt
                filterInput {
                    val text = message.lowercase()
                    if (!text.contains("task") && !text.contains("list") && !text.contains("add") && !text.contains("remove")) {
                        // Interrupt processing and return the message "I only handle tasks."
                        breakWith("I only handle tasks.")
                    }
                }

                // Connect the tools (functions) that the agent can call
                tools {
                    +"add_task"
                    +"remove_task"
                    +"list_tasks"
                }
            }
        }

        // Define the function implementations (Tools)
        defineFunctions {
            // To persist tasks between calls, we can use either:
            // - memory(AgentMemoryType.SHORT_TERM)
            // - a global mutable variable (simpler for demos)
            val tasks = mutableListOf<String>()

            function(
                name = "add_task",
                description = "Add a task to the list",
                params = types(string("description", "The description of the new task."))
            ) { (description) ->
                tasks.add(description as String)
                "Task '$description' added. Now you have ${tasks.size} task(s)."
            }

            function(
                name = "remove_task",
                description = "Remove a task by its exact name",
                params = types(string("description", "The task to remove."))
            ) { (description) ->
                val removed = tasks.removeIf { it.equals(description as? String, ignoreCase = true) }
                if (removed) "Task '$description' removed."
                else "No such task found: '$description'."
            }

            function(
                name = "list_tasks",
                description = "List all tasks currently stored",
                params = types()
            ) {
                if (tasks.isEmpty()) {
                    "No tasks found."
                } else {
                    "Here are your tasks:\n" + tasks.joinToString("\n") { "- $it" }
                }
            }
        }
    }

In [5]:
import org.eclipse.lmos.arc.agents.ChatAgent
import org.eclipse.lmos.arc.agents.getAgentByName

// Get the task manager agent instance we defined earlier
val agent = agentBuilder.getAgentByName("task-manager") as? ChatAgent ?: error("Agent not found!")

# Send a task list

In [6]:
import org.eclipse.lmos.arc.agents.User
import org.eclipse.lmos.arc.agents.conversation.Conversation
import org.eclipse.lmos.arc.agents.conversation.UserMessage
import org.eclipse.lmos.arc.core.getOrNull

val conversation = Conversation(User("demoUser"))

// Посмотрим на некоторые сообщения
val messages = listOf(
    "Hi, I'm new here. Can you help me organize my tasks?",
    "I need to add a task: Buy groceries for dinner",
    "Add another task: Complete the quarterly report by Friday",
    "Add task: Schedule team meeting for next week",
    "Can you show me all my current tasks?",
    "I finished buying groceries - please remove that task",
    "What tasks do I still have pending?",
    "Could you tell me today's weather forecast?"
)

val exchanges = mutableListOf<Pair<String, String?>>()

In [7]:
// Run the conversation and generate HTML
runBlocking {
    var tempConv = conversation

    messages.forEach { userMessage ->
        // Add the user message to the conversation
        tempConv += UserMessage(userMessage)

        // Have the agent process the updated conversation
        val result = agent.execute(tempConv).getOrNull()

        // Extract the agent's reply (last message in transcript)
        val agentReply = result?.transcript?.lastOrNull()?.content

        // Print the exchange in console
        println("User: $userMessage")
        println("Agent: $agentReply")
        println("----------\n")

        // Save the exchange for HTML generation
        exchanges.add(userMessage to agentReply)

        // Update the conversation for the next iteration
        if (result != null) tempConv = result
    }
}

User: Hi, I'm new here. Can you help me organize my tasks?
Agent: Of course! I'd be glad to help. You can tell me to add a new task, remove a completed one, or list all your current tasks. What would you like to do first?
----------

User: I need to add a task: Buy groceries for dinner
Agent: Great! Your task to "Buy groceries for dinner" has been added. Is there anything else you'd like to add or perhaps another action you'd like to perform?
----------

User: Add another task: Complete the quarterly report by Friday
Agent: Your task to "Complete the quarterly report by Friday" has been added. Now you have a total of 2 tasks. Can I assist you with anything else?
----------

User: Add task: Schedule team meeting for next week
Agent: Your task to "Schedule team meeting for next week" has been added. Now you have a total of 3 tasks. Is there anything else I can assist you with?
----------

User: Can you show me all my current tasks?
Agent: Here are your current tasks:

1. Buy groceries fo

# Generate HTML

In [8]:
// Function to generate minimal HTML for the conversation
fun generateMinimalHtml(exchanges: List<Pair<String, String?>>): String {
    val sb = StringBuilder()

    // HTML header with dark mode compatibility
    sb.append("""
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset="UTF-8">
      <title>Task Manager Chat</title>
      <style>
        body {
          font-family: system-ui, sans-serif;
          max-width: 600px;
          margin: 20px auto;
          padding: 0 15px;
          color: inherit;
        }
        .chat {
          display: flex;
          flex-direction: column;
          gap: 8px;
        }
        .msg {
          padding: 8px 12px;
          border-radius: 6px;
          max-width: 80%;
          color: inherit;
        }
        .user {
          align-self: flex-end;
          background: rgba(100, 180, 255, 0.15);
          border: 1px solid rgba(100, 180, 255, 0.3);
        }
        .agent {
          align-self: flex-start;
          background: rgba(200, 200, 200, 0.1);
          border: 1px solid rgba(200, 200, 200, 0.2);
        }
        .divider {
          border-top: 1px solid rgba(200, 200, 200, 0.1);
          margin: 8px 0;
        }
      </style>
    </head>
    <body>
      <h2>Task Manager Conversation</h2>
      <div class="chat">
    """.trimIndent())

    // The rest of your function stays the same
    exchanges.forEachIndexed { index, (userMessage, agentReply) ->
        // User message
        sb.append("    <div class=\"msg user\">")
        sb.append(userMessage.replace("\n", "<br>"))
        sb.append("</div>\n")

        // Agent reply
        sb.append("    <div class=\"msg agent\">")
        sb.append(agentReply?.replace("\n", "<br>") ?: "")
        sb.append("</div>\n")

        // Add divider between exchanges (except after the last one)
        if (index < exchanges.size - 1) {
            sb.append("    <div class=\"divider\"></div>\n")
        }
    }

    // HTML footer
    sb.append("""
      </div>
    </body>
    </html>
    """.trimIndent())

    return sb.toString()
}

In [9]:
// Generate and save HTML
val html = generateMinimalHtml(exchanges)
HTML(html)

<!DOCTYPE html>
 
 
 
 Task Manager Chat 
 
 
 
 Task Manager Conversation 
 Hi, I'm new here. Can you help me organize my tasks? 
 Of course! I'd be glad to help. You can tell me to add a new task, remove a completed one, or list all your current tasks. What would you like to do first? 
 
 I need to add a task: Buy groceries for dinner 
 Great! Your task to "Buy groceries for dinner" has been added. Is there anything else you'd like to add or perhaps another action you'd like to perform? 
 
 Add another task: Complete the quarterly report by Friday 
 Your task to "Complete the quarterly report by Friday" has been added. Now you have a total of 2 tasks. Can I assist you with anything else? 
 
 Add task: Schedule team meeting for next week 
 Your task to "Schedule team meeting for next week" has been added. Now you have a total of 3 tasks. Is there anything else I can assist you with? 
 
 Can you show me all my current tasks? 
 Here are your current tasks: 1. Buy groceries for dinner 2. Complete the quarterly report by Friday 3. Schedule team meeting for next week Do you need help with anything else? 
 
 I finished buying groceries - please remove that task 
 The task "Buy groceries for dinner" has been successfully removed from your list. Do you have any other tasks that you would like to manage? 
 
 What tasks do I still have pending? 
 Here are your remaining tasks: 1. Complete the quarterly report by Friday 2. Schedule team meeting for next week Do you need to update any other tasks? 
 
 Could you tell me today's weather forecast? 
 I only handle tasks.